In [66]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import misc
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import cv2
from sklearn.model_selection import train_test_split
K.set_image_dim_ordering('th')
%matplotlib nbagg

In [61]:
img_height, img_width = 480,640
from keras.preprocessing import image
from itertools import izip

batch_size = 1
def hot_vectorize(x):
    zero_mask = x==0
    non_zero_mask = x!=0
    x[zero_mask] = 1
    x[non_zero_mask] = 0
    return x

labeldatagen = image.ImageDataGenerator(
        #preprocessing_function=applications.xception.preprocess_input,
        #preprocessing_function = random_crop,
        #zoom_range=.5,
        preprocessing_function = hot_vectorize,
        data_format="channels_last")

datagen = image.ImageDataGenerator(
        #preprocessing_function=applications.xception.preprocess_input,
        #preprocessing_function = random_crop,
        rescale =1/500.,
        #zoom_range=.5,
        data_format="channels_last")

flow_from_directory_params = {'target_size': (img_height,img_width),
                                  'color_mode': 'grayscale',
                                  'class_mode': None,
                                  'batch_size': batch_size}

mask_generator = labeldatagen.flow_from_directory(
        directory="/media/drc/DATA/chris_labelfusion/logs_test/test_gan/",
        shuffle =False,
        **flow_from_directory_params
    )
'''
rgb_generator = datagen.flow_from_directory(
        directory="/media/drc/DATA/chris_labelfusion/logs_test/test_gan_sim/",
        **flow_from_directory_params
    )
normal_generator = datagen.flow_from_directory(
        directory="/media/drc/DATA/chris_labelfusion/logs_test/test_gan_sim/",
        **flow_from_directory_params
    )
'''
gt_generator = datagen.flow_from_directory(
        directory="/media/drc/DATA/chris_labelfusion/logs_test/test_gan_sim/",
        shuffle =False,
        **flow_from_directory_params
    )
train_generator =  izip(gt_generator, mask_generator)


'''
plt.figure()
plt.imshow(np.reshape(depth_generator.next()[1],(img_height,img_width)))
plt.figure()
plt.imshow(np.reshape(gt_generator.next()[1],(img_height,img_width)))
'''

Found 3978 images belonging to 1 classes.
Found 3978 images belonging to 1 classes.


'\nplt.figure()\nplt.imshow(np.reshape(depth_generator.next()[1],(img_height,img_width)))\nplt.figure()\nplt.imshow(np.reshape(gt_generator.next()[1],(img_height,img_width)))\n'

In [67]:
#X_train, X_test, y_train, y_test = train_test_split(
     #X, y, test_size=0.20, random_state=42)
#keras.layers.Cropping2D(cropping=((0, 0), (0, 0)), data_format=None)
#keras.layers.GaussianNoise(stddev)

model = Sequential()
model.add(Convolution2D(32, (3, 3), border_mode='same', input_shape=(batch_size,img_height,img_width,1)))
model.add(Activation('relu'))

model.add(Convolution2D(32, (3, 3), border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(64, (3, 3), border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(64, (3, 3), border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(1, 1, 1, border_mode='same'))

model.add(Reshape((480, 640)))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['mae', 'acc'])
print(model.summary())
model.fit_generator(
        train_generator,
        steps_per_epoch=2,
        epochs=50)

model.save_weights('first_try.h5') 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", input_shape=(1, 480, 6...)`
  import sys


ValueError: Input 0 is incompatible with layer conv2d_1: expected ndim=4, found ndim=5